In [ ]:
import sys
import os
from pathlib import Path

# 将src目录添加到系统路径中
sys.path.append('../')

# 导入本地模块
#from src.timellm import TimeLLM
from src.baselines import TimeLLM 
from src.baselines import PatchTST
from data_provider.data_factory import data_provider
from utils.tools import *

# 导入第三方库
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
#import tqdm
from tqdm import tqdm
import time
from accelerate import Accelerator, DeepSpeedPlugin
from accelerate import DistributedDataParallelKwargs

os.environ['CURL_CA_BUNDLE'] = ''
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:64"

In [ ]:
# 数据相关参数
DATA_NAME = 'electricity/electricity.csv'
DESCRIPTION_PATH = '../dataset/prompt_bank/ECL.txt'
DATA_PATH = Path('../dataset') / DATA_NAME

# 模型相关参数
TARGET_COL = 'OT'       # 目标列，代表电力需求
NUM_EPOCHS = 3         # 训练轮数
LEARNING_RATE = 1e-4    # 学习率
DEVICE = 'cuda:1'

# 加载描述文本
try:
    with open('../dataset/prompt_bank/ECL.txt', 'r', encoding='utf-8') as f:
        description = f.read().strip()
except FileNotFoundError:
    print("Failed to find ECL.txt.")
    description = ""

class Configs:
    def __init__(self, task_name='long_term_forecast', 
                 pred_len=96, seq_len=96, d_ff=16,devices="1,2,3",use_multi_gpu=True,
                 d_model=8, patch_len=16, stride=8, llm_model='GPT2',
                 llm_layers=6, prompt_domain=0, content=description,
                 dropout=0.1, enc_in=7, dec_in=7, c_out=7, n_heads=3,
                 llm_dim=768,data='custom', train_epochs=NUM_EPOCHS, root_path="../dataset/", data_path=DATA_NAME,
                 embed='timeF',batch_size=32, freq='h', label_len=48, features='M',
                 target=TARGET_COL, seasonal_patterns='Monthly', augmentation_ratio=0,
                 num_workers=3, patience=3, output_attention=False, use_amp=False, **kwconfig):
        self.task_name = task_name
        self.pred_len = pred_len
        self.seq_len = seq_len
        self.d_ff = d_ff
        self.d_model = d_model
        self.patch_len = patch_len
        self.stride = stride
        self.llm_model = llm_model
        self.llm_layers = llm_layers
        self.prompt_domain = prompt_domain
        self.content = content
        self.dropout = dropout
        self.enc_in = enc_in
        self.dec_in = dec_in
        self.c_out = c_out
        self.n_heads = n_heads
        self.llm_dim = llm_dim
        self.data = data
        self.root_path = root_path
        self.data_path = data_path
        self.embed = embed
        self.batch_size = batch_size
        self.freq = freq
        self.label_len = label_len
        self.features = features
        self.target = target
        self.seasonal_patterns = seasonal_patterns
        self.augmentation_ratio = augmentation_ratio
        self.num_workers = num_workers
        self.patience = patience
        self.output_attention = output_attention
        self.use_amp = use_amp
        self.devices = devices
        self.use_multi_gpu = use_multi_gpu  
        self.train_epochs = train_epochs     
        self.__dict__.update(kwconfig)

config = Configs()
config.use_gpu = True if torch.cuda.is_available() else False

if config.use_gpu and config.use_multi_gpu:
    config.devices = config.devices.replace(' ', '')
    device_ids = config.devices.split(',')
    config.device_ids = [int(id_) for id_ in device_ids]
    config.gpu = config.device_ids[0]

In [ ]:
# 加载数据集，解析 'date' 列为日期格式，并设置为索引
try:
    data = pd.read_csv(DATA_PATH, parse_dates=['date'], index_col='date')
    print(data.head())
except FileNotFoundError:
    print(f"Failed to find {DATA_PATH}.")
    data = pd.DataFrame()

# 检查必要的列是否存在
if TARGET_COL not in data.columns:
    raise ValueError(f"Not found '{TARGET_COL}' in the dataset. Please check.")

# 数据标准化
scaler = StandardScaler()
feature_cols = ['OT']  # 根据需要添加更多特征列
scaled_data = scaler.fit_transform(data[feature_cols].values)
scaled_data = pd.DataFrame(scaled_data, index=data.index, columns=feature_cols)

print(scaled_data.head())

In [ ]:
# 提取最后 SEQ_LEN 个时间步的数据
time_series_data = scaled_data[-config.seq_len:]

# 转换为张量
time_series_tensor = torch.tensor(time_series_data.values, dtype=torch.float32).to(DEVICE)

print("Standarded Time series data shape: ", time_series_tensor.shape)

# 绘制前5个特征的时间序列
plt.figure(figsize=(12, 6))
for i, col in enumerate(feature_cols[:5]):
    plt.plot(time_series_data.index, time_series_data[col], label=col)
plt.title("Time series data")
plt.xlabel("Date")
plt.ylabel("Standardized value")
plt.legend()
plt.show()

In [ ]:
def create_windows(data, target_col, seq_len, horizon):
    """
    使用滑动窗口方法创建输入和目标数据。
    
    参数:
    - data (DataFrame): 输入数据
    - target_col (str): 目标列名
    - seq_len (int): 输入序列长度
    - horizon (int): 预测步数
    
    返回:
    - X (ndarray): 输入特征
    - y (ndarray): 目标值
    """
    X = []
    y = []
    for i in range(len(data) - seq_len - horizon + 1):
        X.append(data.iloc[i:i+seq_len].values)
        y.append(data.iloc[i+seq_len:i+seq_len+horizon][target_col].values)
    return np.array(X), np.array(y)

# 创建滑动窗口
X, y = create_windows(scaled_data, TARGET_COL, config.seq_len, config.pred_len)
print("滑动窗口后的 X 形状:", X.shape)  # [样本数, SEQ_LEN, 特征数]
print("滑动窗口后的 y 形状:", y.shape)  # [样本数, HORIZON]

# 分割训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

print("训练集形状:", X_train.shape, y_train.shape)
print("测试集形状:", X_test.shape, y_test.shape)

In [ ]:
# 选择一个批次进行测试
sample_time_series = X_train[:config.batch_size]
sample_time_series_tensor = torch.tensor(sample_time_series, dtype=torch.float32).to(DEVICE)

In [ ]:
# 定义预测层
prediction_layer = nn.Linear(config.d_model, 1).to(DEVICE)  # 单变量预测

# 实例化 TimeLLM 模块
#time_llm = TimeLLM.Model(config).float().to(DEVICE)
model = TimeLLM.Model(config).float().to(DEVICE)
print("TimeLLM 模块已实例化。")

# class TimeSeriesPredictor(nn.Module):
#     """
#     时间序列预测器模型，结合 TimeLLM 模块和预测层。
#     """
#     def __init__(self, time_llm, prediction_layer):
#         super(TimeSeriesPredictor, self).__init__()
#         self.time_llm = time_llm
#         self.prediction_layer = prediction_layer
    
#     def forward(self, time_series_data, description, pred_len, seq_len):
#         """
#         前向传播函数。
        
#         参数:
#         - time_series_data (Tensor): 输入的时间序列数据，形状 [B, L, D]
#         - description (str): 描述文本
#         - pred_len (int): 预测步数
#         - seq_len (int): 序列长度
        
#         返回:
#         - predictions (Tensor): 预测结果，形状 [B, pred_len, 1]
#         """
#         multi_modal_embedding, _ = self.time_llm(
#             time_series_data, description, pred_len, seq_len
#         )
#         predictions = self.prediction_layer(multi_modal_embedding)
#         return predictions

# # 实例化完整模型
# model = TimeSeriesPredictor(time_llm, prediction_layer).to(DEVICE)
# model.train()
# print("完整时间序列预测模型已实例化并切换到训练模式。")

In [ ]:
ddp_kwconfig = DistributedDataParallelKwargs(find_unused_parameters=True)
deepspeed_plugin = DeepSpeedPlugin(hf_ds_config='./ds_config_zero2.json')
accelerator = Accelerator(kwargs_handlers=[ddp_kwconfig], deepspeed_plugin=deepspeed_plugin)

train_data, train_loader = data_provider(config, 'train')
vali_data, vali_loader = data_provider(config, 'val')
test_data, test_loader = data_provider(config, 'test')
# 定义损失函数和优化器
criterion = nn.MSELoss()
model_optim = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(model_optim, T_max=20, eta_min=1e-8)

train_loader, vali_loader, test_loader, model, model_optim, scheduler = accelerator.prepare(train_loader, vali_loader, test_loader, model, model_optim, scheduler)

In [ ]:
train_steps = len(train_loader)
early_stopping = EarlyStopping(accelerator=accelerator, patience=config.patience)
mae_metric = nn.L1Loss()
time_now = time.time()
#if config.use_multi_gpu and config.use_gpu:
#    model = nn.DataParallel(model, device_ids=config.device_ids)

for epoch in range(NUM_EPOCHS):
    iter_count = 0
    train_loss = []

    model.train()
    epoch_time = time.time()
    for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in tqdm(enumerate(train_loader)):
        iter_count += 1
        model_optim.zero_grad()

        batch_x = batch_x.float().to(accelerator.device)
        batch_y = batch_y.float().to(accelerator.device)
        batch_x_mark = batch_x_mark.float().to(accelerator.device)
        batch_y_mark = batch_y_mark.float().to(accelerator.device)

        # decoder input
        dec_inp = torch.zeros_like(batch_y[:, -config.pred_len:, :]).float().to(
            accelerator.device)
        dec_inp = torch.cat([batch_y[:, :config.label_len, :], dec_inp], dim=1).float().to(
            accelerator.device)

        # encoder - decoder
        if config.use_amp:
            with torch.cuda.amp.autocast():
                if config.output_attention:
                    outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
                else:
                    outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

                f_dim = -1 if config.features == 'MS' else 0
                outputs = outputs[:, -config.pred_len:, f_dim:]
                batch_y = batch_y[:, -config.pred_len:, f_dim:].to(accelerator.device)
                loss = criterion(outputs, batch_y)
                train_loss.append(loss.item())
        else:
            if config.output_attention:
                outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)[0]
            else:
                outputs = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)

            f_dim = -1 if config.features == 'MS' else 0
            outputs = outputs[:, -config.pred_len:, f_dim:]
            batch_y = batch_y[:, -config.pred_len:, f_dim:]
            loss = criterion(outputs, batch_y)
            train_loss.append(loss.item())

        if (i + 1) % 200 == 0:
            accelerator.print(
                "\titers: {0}, epoch: {1} | loss: {2:.7f}".format(i + 1, epoch + 1, loss.item()))
            speed = (time.time() - time_now) / iter_count
            left_time = speed * ((config.train_epochs - epoch) * train_steps - i)
            accelerator.print('\tspeed: {:.4f}s/iter; left time: {:.4f}s'.format(speed, left_time))
            iter_count = 0
            time_now = time.time()

        if config.use_amp:
            scaler.scale(loss).backward()
            scaler.step(model_optim)
            scaler.update()
        else:
            accelerator.backward(loss)
            model_optim.step()

    accelerator.print("Epoch: {} cost time: {}".format(epoch + 1, time.time() - epoch_time))
    train_loss = np.average(train_loss)
    vali_loss, vali_mae_loss = vali(config, accelerator, model, vali_data, vali_loader, criterion, mae_metric)
    test_loss, test_mae_loss = vali(config, accelerator, model, test_data, test_loader, criterion, mae_metric)
    accelerator.print(
        "Epoch: {0} | Train Loss: {1:.7f} Vali Loss: {2:.7f} Test Loss: {3:.7f} MAE Loss: {4:.7f}".format(
            epoch + 1, train_loss, vali_loss, test_loss, test_mae_loss))

    path = os.path.join(accelerator.exp_dir, 'best.pth')
    early_stopping(vali_loss, model, path)
    if early_stopping.early_stop:
        accelerator.print("Early stopping")
        break

    if config.lradj != 'TST':
        if config.lradj == 'COS':
            scheduler.step()
            accelerator.print("lr = {:.10f}".format(model_optim.param_groups[0]['lr']))
        else:
            if epoch == 0:
                config.learning_rate = model_optim.param_groups[0]['lr']
                accelerator.print("lr = {:.10f}".format(model_optim.param_groups[0]['lr']))
            adjust_learning_rate(accelerator, model_optim, scheduler, epoch + 1, config, printout=True)

    else:
        accelerator.print('Updating learning rate to {}'.format(scheduler.get_last_lr()[0]))


In [ ]:
# 保存模型状态字典
model_save_path = 'time_series_predictor.pth'
torch.save(model.state_dict(), model_save_path)
print(f"模型已保存到 {model_save_path}。")

# 加载模型（示例）
# model.load_state_dict(torch.load(model_save_path))
# model.eval()
# print("模型已加载并切换到评估模式。")